# Summarization

This is a very high-level task that summarizes the major themes of a set of feedback comments.

## Imports and setup

In [2]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [3]:
import pandas as pd
from pprint import pprint
from IPython.display import Markdown, display
from pathlib import Path
from feedback_analyzer.summarization import summarize_comments
from feedback_analyzer.models_common import LLMConfig

In [4]:
# this makes it more robust to run async tasks inside an already async environment (jupyter notebooks)
import nest_asyncio
nest_asyncio.apply()

Make sure to either set `ANTHROPIC_API_KEY` as an environment variable or put it in a .env file and use the following cell to load the env var. The format in the .env file is:
```
ANTHROPIC_API_KEY=yourKeyGoesHere
```

In [5]:
%load_ext autoreload
%autoreload 2

This is a convenience function to make seeing Pandas dataframe values easier, especially when there are long strings like the student comments we will be using.

In [6]:
def full_show(df):
    with pd.option_context('display.max_columns', None, 'display.max_rows', None, 'display.max_colwidth', None):
        display(df)

This is a convenience function for pretty-printing long student comments.

In [7]:
def print_wrap(text: str, width: int = 72) -> str:
    print(textwrap.fill(text, width=width))

In [8]:
MODEL_NAME_HAIKU = "claude-3-haiku-20240307"

## Load the example data

In [9]:
data_path = Path('../data/example_data')

Let's load up some fake data. 

All of these comments are synthetic to avoid sharing any sensitive or PII information, but they should work great for illustration purposes. There are 100 rows, with just a few null/nan values here and there for realism. In most surveys I've seen, there are quite a number of null/None/blank etc values, and the functions are written to handle those.

In [10]:
example_survey = pd.read_csv(data_path / 'example_survey_data_synthetic.csv')
full_show(example_survey.head())

,best_parts,enhanced_learning,improve_course
0,I valued the practical clinical aspects related to immune-related disorders and their management.,The illustrative visuals and straightforward explanatory clips.,Consider reducing the duration of certain videos. A few appeared to be slightly prolonged.
1,The flexibility to learn at a self-determined speed,The opportunity to review the lecture content,"The pace of some lectures could be slowed down. At times, it's challenging to follow the lecturer's speech or decipher their handwriting."
2,The educational content was extremely enriching and stimulating! The section on oncology was the highlight.,the self-assessment activities.,Nothing specific comes to mind.
3,Professional growth within the medical sector,"The practical integration workshops were highly beneficial, they significantly contributed to a deeper comprehension of the theories and their implementation in a healthcare environment.",Incorporating a few advanced projects as optional tasks could benefit learners who wish to delve deeper into the subject matter. These projects wouldn't need to influence exam scores.
4,The highlights of the class included the practical demonstration clips that made the complex biological principles more understandable by connecting them to daily well-being and actions. This connection was incredibly beneficial as I navigated the course content.,"The aspect of the course that most facilitated my learning was the regular assessments provided at each segment, which helped confirm my grasp of the material presented. These checkpoints effectively guided me in the correct learning direction. It's evident that considerable effort was invested in designing these educational modules to enable students to gain a deep comprehension rather than just a superficial understanding of the subject matter.","Extend the duration of the concept videos for the more challenging topics, as they require a deeper dive to fully grasp the intricacies involved. Additionally, consider introducing an additional educator to the mix. The dynamic of having multiple voices in another subject area is quite engaging, and it would be beneficial to replicate that experience in this subject to prevent monotony from setting in with just one instructor."


## Summarization

Here we are having the model summarize the major themes of the feedback comments based on the survey question. Given that the models have long context windows (200K for the Claude models), we just stuff all of the comments together into the prompt. If you change to use Haiku (see commented out line) - a small but capable model, just be aware that it often comes back with significantly fewer themes than what Sonnet 3.5 (the default we are using) would return.

In [13]:
improve_course_question = "What could be improved about the course?"
comments = example_survey['improve_course'].tolist() # 100 comments
summarization_result = await summarize_comments(comments=comments, 
                                                question=improve_course_question)
                                                # llm_config=LLMConfig(model=MODEL_NAME_HAIKU))

display(Markdown((summarization_result.summary)))

The major themes of feedback shared by the students for course improvement include:

1. Video content:
   - Some students suggested reducing the duration of certain videos, while others requested longer videos for complex topics.
   - The pace of lectures was sometimes too fast, making it difficult to follow.
   - More visual aids and multimedia presentations were requested to enhance understanding.
   - Improving the clarity of video lectures was mentioned.

2. Course content and depth:
   - Many students expressed a desire for more in-depth content, especially on advanced topics.
   - Requests for additional subjects, such as renal physiology, endocrinology, immunology, and next-generation sequencing techniques.
   - Some students wanted more practical examples, case studies, and real-life applications.
   - Suggestions to include more content on innovative therapies and recent advancements in treatments.

3. Interactive elements and practical applications:
   - Requests for more hands-on activities and interactive elements.
   - Suggestions for incorporating simulation tools or projects to apply learned concepts.
   - Desire for more practical clinical demonstrations and patient scenarios.

4. Assessment and quizzes:
   - Some students found quiz questions confusing or not well-aligned with the course material.
   - Requests for additional mock tests and practice questions for the final assessment.
   - Suggestions to increase the number of attempts for the final assessment.
   - Proposal to include a mid-term assessment to identify areas needing further study.

5. Course materials and resources:
   - Requests for downloadable and comprehensive course materials (e.g., PDF format).
   - Suggestions for improved note-taking spaces and more detailed outlines.
   - Desire for ongoing access to course content for future reference.

6. Course structure and duration:
   - Some students felt the course was too short and wanted it extended.
   - Suggestions to include optional advanced projects for deeper learning.

7. Language and consistency:
   - Requests for more consistent language use across different presenters.
   - Suggestions for clearer and more straightforward phrasing of questions.
   - Requests for subtitles in other languages to assist non-native English speakers.

8. Positive feedback:
   - Many students expressed satisfaction with the course, describing it as excellent or outstanding.
   - Some students stated that no improvements were necessary.

These themes reflect a diverse range of student experiences and preferences, with some contradictory feedback (e.g., video length) highlighting the challenge of meeting all individual needs in a single course format.

This summary gives a pretty reasonable sense at a high level of what students said for this survey question. But if you're looking for more structured output (something you could use as categories in a bar graph, for example), then check out the example notebook on theme derivation, or the (much longer) notebook with the end-to-end example.

Let's now try for a different survey question:

In [14]:
comments2 = list(reversed(example_survey['improve_course'].tolist())) # 100 comments
summarization_result2 = await summarize_comments(comments=comments2, 
                                                question=improve_course_question)
                                                # llm_config=LLMConfig(model=MODEL_NAME_HAIKU))

display(Markdown((summarization_result2.summary)))


Based on the student feedback comments, the major themes for course improvement are:

1. Additional Learning Resources:
   - Many students requested more comprehensive study materials, including downloadable PDFs, expanded notes, and additional visual aids.
   - Some suggested incorporating more multimedia presentations and interactive elements.
   - There were requests for more practice questions, mock tests, and case studies.

2. Content Depth and Breadth:
   - Several students expressed a desire for deeper exploration of certain topics, including renal physiology, endocrinology, immunology, and next-generation sequencing.
   - Some suggested broadening the curriculum to cover more contemporary topics and innovative therapies.
   - There were requests for more practical applications, real-world scenarios, and clinical demonstrations.

3. Video Lecture Improvements:
   - Some students found the pace of lectures too fast, suggesting slower delivery or the ability to acknowledge speed changes.
   - There were requests for clearer video presentations and improved audio quality.
   - Some suggested incorporating more visual aids in the lectures.

4. Assessment Alignment:
   - Several students noted that quiz and exam questions sometimes covered material not thoroughly discussed in the course.
   - There were suggestions to better align assessments with the course content and learning objectives.

5. Accessibility and User Experience:
   - Students requested selectable subtitles for copying, pagination in study resources, and subtitles in multiple languages.
   - Some suggested improving the readability of course materials, including font selection.

6. Interactive and Practical Components:
   - Many students expressed a desire for more hands-on activities, practical exercises, and interactive elements.
   - Some suggested incorporating small-scale projects or simulations to apply learned concepts.

7. Content Organization and Consistency:
   - There were suggestions to ensure consistency in language and presentation across different instructors and videos.
   - Some students requested better organization of discussion forums and question submission features.

8. Extended Access and Course Duration:
   - Several students expressed a desire for ongoing access to course materials after completion.
   - Some suggested extending the course duration to allow for deeper exploration of topics.

9. Advanced Content:
   - A few students requested optional advanced projects or sections for those wanting to delve deeper into the subject matter.

10. Positive Feedback:
    - Many students expressed satisfaction with the course, describing it as excellent or outstanding.
    - Some stated that no improvements were necessary.

These themes reflect a desire for more comprehensive and diverse learning resources, deeper content exploration, improved alignment between course material and assessments, enhanced interactivity, and better accessibility. While many students were satisfied with the course, there's a clear appetite for more advanced and practical applications of the material.

This may be good enough. If we want to have the output in the form of a list of themes (that is, more structured), we could use a model with a tool call to extract into a more structured format, or we could use the theme_derivation function in the `theme_derivation_example.ipynb` notebook. Check that out next...